## Visualizing the results using tables and plots

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [4]:
nvda_data1_df = pd.read_csv("NVDA_outputData1.csv")
spy_data1_df = pd.read_csv("SPY_outputData1.csv")
vix_data1_df = pd.read_csv("^VIX_outputData1.csv")

In [18]:
df= pd.read_csv("options_data1.csv")
nvda_data1_spot = df.query('ticker == "NVDA"').iloc[1,-2]
spy_data1_spot = df.query('ticker == "SPY"').iloc[1,-2]

In [22]:
nvda_data1_spot


131.694397

In [20]:
nvda_strikes = nvda_data1_df["Strike"].values
# Find index of the closest strike
closest_index = np.argmin(np.abs(nvda_strikes - nvda_data1_spot))

# Get the closest strike using the index
closest_strike = nvda_strikes[closest_index]

In [36]:
#creating a empty df
nvda_iv_table = pd.DataFrame()
for expiration in nvda_data1_df["Expiration"].unique():
    expiration_df= nvda_data1_df[nvda_data1_df["Expiration"] == expiration]
    nvda_strikes = expiration_df["Strike"].values
    # Find index of the closest strike
    closest_index = np.argmin(np.abs(nvda_strikes - nvda_data1_spot))

    # Get the closest strike using the index
    closest_strike = nvda_strikes[closest_index]
    for index, row in expiration_df.iterrows():
        if row.iloc[3] == closest_strike:
        # filter only necessary columns from the row and insert it into df
            desired_columns = [ "Expiration", "OptionType", "Strike", "BisectionIV", "NewtonIV", "ImpliedVolatility"]
            nvda_iv_table = pd.concat([nvda_iv_table, pd.DataFrame([row[desired_columns]])], ignore_index=True)


In [37]:
nvda_iv_table

,Expiration,OptionType,Strike,BisectionIV,NewtonIV,ImpliedVolatility
0,2025-02-21,Call,132.0,0.458217,0.458217,0.419684
1,2025-02-21,Put,132.0,0.511800,0.511800,0.453130
2,2025-03-21,Call,132.0,0.586375,0.586375,0.585942
3,2025-03-21,Put,132.0,0.611904,0.611904,0.578007
4,2025-04-17,Call,130.0,0.532450,0.532450,0.545293
5,2025-04-17,Put,130.0,0.548353,0.548353,0.517461
